# Elektronische Bauelemente: MOS-Transistor

Dies ist eine interaktive Oberfläche zur Erkundung verschiedener Zusammenhänge des MOS-Transistors auf Basis der Vorlesung

In [1]:
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

from cycler import cycler
from matplotlib.lines import Line2D 

from ipywidgets import (
    HBox, VBox, FloatText, FloatRangeSlider, IntSlider, FloatSlider, 
    Play, jslink, FloatLogSlider, interactive_output, HTML, HTMLMath, Layout, Checkbox, Label, Text
)


In [2]:
# global constants
eps0        = 8.854e-12     # [As/(Vm)] - permittivity of free space
q           = 1.602e-19     # [As]      - electronic charge
kb          = 1.38e-23      # [J/K]     - Boltzmann's constant

# global fixed parameters
T           = 300           # [K]       - temperature
UT          = kb*T/q        # [V]       - thermal voltage
epsr_si     = 11.7          # [ ]       - relative permittivity of Si
epsr_ox     = 3.73          # [ ]       - relative permittivity of SiO2
ni          = 1.07e10       # [cm^-3]    - intrinsic carrier concentration in Si (at 300 K)
Lch         = 0.5e-6        # [m]       - channel length
mu_n        = 0.1           # [m^2/(Vs)]- mobility of electrons in Si
Uth0        = 0.7           # [V]       - threshold voltage
UA          = 50            # [V]       - early voltage

# Bias point initialization
Ugs_sweep   = np.linspace(0,2.5,101)     # [V]   - gate-source voltage
Uds_sweep   = np.linspace(0,5,101)      # [V]   - drain-source voltage
Usb_sweep   = np.linspace(0,5,101)      # [V]   - source-substrate voltage

### Widget definitions
# --------------------
Uth0_wid = Label(value=r'Schwell Spannung \(U_{\rm{th0}}\)'+' = '+str(Uth0)+' V') # threshold voltage - Uth0
Lch_wid = Label(value=r'Kanallänge \(L\)'+' = '+str(Lch*1e6)+' µm') # Channel length - L
mu_n_wid = Label(value=r'El. Beweglichkeit \(\mu_{\rm{n}}(N_{\rm{A}}^{-})\)'+' = '+str(mu_n*1e4)+r' \(\rm{cm^{2}/(Vs)}\)') # Electron mobility 
ni_wid = Label(value=r'Eigenleitungsdichte  \(n_{\rm{i}}\)'+' = '+str(ni/1e10)+r' \(\rm{\times 10^{10} cm^{-3}}\)') # Intrinsic concentration
UA_wid = Label(value=r'Early Spannung \(U_{\rm{af}}\)'+' = '+str(UA)+' V') # Early voltage
temp_wid = Label(value=r'Temperatur \(T\)'+' = '+str(T)+' K') # Temperature

### Equations
# -----------
IDeqn_title = HTMLMath(
    value=r"Ausgangsgleichungen (n-kanal-Enhancement MOSFET): $I_{\mathrm{D}}=0 \mathrm{\;für\;} U_{\mathrm{GS}} < U_{\mathrm{th}}$")
IDeqn_lin = HTMLMath(
    value=r"Linearer Berich$$\displaystyle I_{\mathrm{D,lin}} = k'_{n}\left[\frac{1-k_{clm}}{2(1+a_{th})}(U_{GS}-U_{th})^2 + \frac{k_{clm}}{2}(U_{GS}-U_{th})U_{DS}\right]$$", # \text{for $U_{GS}<U_{th},U_{DS}\le U_{DS,sat}$}$$", # \\ I_{\mathrm{D}} = k'_{n}\left[\frac{1-k_{clm}}{2(1+a_{th})}(U_{GS}-U_{th})^2 + \frac{k_{clm}}{2}(U_{GS}-U_{th})U_{DS})\right] \text{for $U_{GS}<U_{th},U_{DS}>U_{DS,sat}$}\\ I_{\mathrm{D}} = 0 \text{ for $U_{GS}<U_{th}$}$$",
)
IDeqn_sat = HTMLMath(
    value=r"Sättigungsberich:$$I_{\mathrm{D,sat}} = k'_{n}\left[(U_{GS}-U_{th})U_{DS} - (1+a_{th})\frac{U_{DS}^2}{2}\right]$$", # \text{for $U_{GS}<U_{th},U_{DS}\le U_{DS,sat}$}$$", # \\ I_{\mathrm{D}} = k'_{n}\left[\frac{1-k_{clm}}{2(1+a_{th})}(U_{GS}-U_{th})^2 + \frac{k_{clm}}{2}(U_{GS}-U_{th})U_{DS})\right] \text{for $U_{GS}<U_{th},U_{DS}>U_{DS,sat}$}\\ I_{\mathrm{D}} = 0 \text{ for $U_{GS}<U_{th}$}$$",
)
kn_ath0_gamma_Cox = HTMLMath(
    value=r"$$k'_{n}=\frac{W}{L}\mu_{n}\bar{C_{ox}},\;a_{th0}=\frac{\gamma}{2\sqrt{\phi_{n}+U_{\mathrm{SB}}}},\;\gamma=\frac{1}{\bar{C_{ox}}}\sqrt{2\epsilon_0 \epsilon_{r,si}qN_{\rm{A}}^{-}},\;\bar{C_{ox}}=\frac{\epsilon_0 \epsilon_{r,ox}}{d_{ox}}$$",
)
Uth_kclm_Udssat = HTMLMath(
    value=r"$$U_{\mathrm{th}}=U_{\mathrm{th0}}+\gamma(\sqrt{\phi_n+U_{\mathrm{SB}}}-\sqrt{\phi_n}),\;k_{clm}=\frac{U_{\mathrm{DS,sat}}}{U_{\mathrm{af}}+U_{\mathrm{DS,sat}}},\;U_{\mathrm{DS,sat}}=\frac{U_{\mathrm{GS}}-U_{\mathrm{th}}}{1+a_{th}}$$",
)

In [3]:
### Style definition 
# ------------------
style = {'description_width': 'initial'}


### Variables and their slider definitions
# ----------------------------------------
# Width (W)
W_slider = FloatSlider(
    value=5,
    min=1,
    max=10,
    step=0.5,
    description=r'\(W/\rm{\mu m}\)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    # style=style
)

# Oxide thickness (d_ox)
dox_slider = FloatSlider(
    value=10,
    min=1,
    max=100,
    step=1,
    description=r'\(d_{\rm{ox}}/\rm{nm}\)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    # style=style
)

# Acceptor concentration (N_A)
NA_slider = FloatLogSlider(
    value=5e16,
    base=10,
    min=15, # max exponent of base
    max=17, # min exponent of base
    step=1e-1, # exponent step
    description=r'\(N_{\rm{A}}^{-}/\rm{cm^{-3}}\)',
    continuous_update=False,
    # style=style
)

# Terminal voltage and its slider definitions
# Gate-source voltage (U_GS) for Output characteristics
Ugs_slider = FloatRangeSlider(
    value=[1, 2.5],
    min=0.5,
    max=4.0,
    step=0.5,
    description='Parameter: $U_{\mathrm{GS}}$/V',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout={'width':'70%'},
    style=style
)
# Drain-source voltage (U_DS) for transfer characteristics
Uds_slider = FloatRangeSlider(
    value=[0.1, 3],
    min=0,
    max=5.0,
    step=0.1,
    description='Parameter: $U_{\mathrm{DS}}$/V',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout={'width':'40%'},
    style=style
)
# source-substrate voltage (U_SB) for plotting Uth vs Usb
Usb_slider = FloatSlider(
    value=0,
    min=np.min(Usb_sweep),
    max=np.max(Usb_sweep),
    step=0.5,
    description='$U_{\mathrm{SB}}$/V',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
#     layout={'width':'40%'}
    # style=style
)

In [4]:
### function definitions
# ----------------------
def calc_Cox(d_ox):
    """Function to oxide capacitance C_ox
    Input
    ------
    d_ox    : float
        oxide thickness.
    Output
    ---------
    C_ox    : float
        oxide capacitance per unit area.
    """
    d_ox_m = d_ox
    C_ox = eps0*epsr_ox/d_ox_m
    return C_ox

def calc_kn(W,L,mu_n):
    """Function to calculate the drain current prefactor.
    Input
    ------
    W       : float 
        gate width.
    L       : float 
        channel length.
    mu_n    : float
        electron mobility.
    Output
    ---------
    km      : float
        drain current prefactor (per unit area).
    """
    C_ox = calc_Cox(dox_slider.value*1e-9)
    
    kn = W/L*mu_n*C_ox
    return kn

def calc_gamma():
    """Function to calculate gamma, i.e. body effect coefficient (see 2.5.19 in [1]).
    Input
    ------
    None
    Output
    ---------
    gamma   : float
        body effect coefficient.
    """
    C_ox = calc_Cox(dox_slider.value*1e-9)
    
    gamma = (1/C_ox)*np.sqrt(2*eps0*epsr_si*q*NA_slider.value*1e6)
    return gamma

def calc_phib():
    """Function to calculate phi_b, i.e. the upper limit of weak inversion (see 2.5.25 in [1]).
    Input
    ------
    None
    Output
    ---------
    phi_b   : float
        the upper limit of weak inversion.
    """
    phi_f = UT*np.log(NA_slider.value/ni)

    phi_b = 2*phi_f
    return phi_b

def calc_ath(Usb):
    """Function to calculate a_th, i.e. the slope of extrapolated threshold voltage Uth vs Usb (see 4.4.33b in [1]).
    Input
    ------
    Usb     : float 
        source-substrate voltage.
    Output
    ---------
    a_th    : float
        slope of extrapolated threshold voltage Uth vs Usb.
    """
    gamma   = calc_gamma()
    phi_b   = calc_phib()

    a_th = 0.5*gamma/np.sqrt(phi_b+Usb)
    return a_th

def calc_Uth(Uth0,Usb):
    """Function to calculate Uth, i.e. gate-source extrapolated threshold voltage (see 4.4.26b in [1]).
    Input
    ------
    Uth0     : float 
        threshold voltage.
    Usb     : float 
        source-substrate voltage.
    Output
    ---------
    Uth    : float
        gate-source extrapolated threshold voltage.
    """
    gamma   = calc_gamma()
    phi_b   = calc_phib()

    Uth = Uth0 + gamma*(np.sqrt(phi_b+Usb) - np.sqrt(phi_b))
    return Uth

def calc_ID(Ugs,Uds,Usb):
    """Function to calculate the drain current ID for single op point (see 4.4.30 in [1]).
    Input
    ------
    Ugs     : float 
        gate-source voltage.
    Uds     : float 
        drain-source voltage.
    Usb     : float 
        source-substrate voltage.
    Output
    ---------
    ID      : float
        Drain current.
    """

    kn      = calc_kn(W_slider.value*1e-6,Lch,mu_n)
    a_th    = calc_ath(Usb)
    Uth     = calc_Uth(Uth0,Usb)
    
    Uds_sat = (Ugs-Uth)/(1+a_th)
    k_clm   = Uds_sat/(UA+Uds_sat)

    if Ugs < Uth:
        ID = 0
    else:
        if Uds <= Uds_sat:
            ID = kn*((Ugs-Uth)*Uds - 0.5*(1+a_th)*Uds**2)
        else:
            ID = kn*((1-k_clm)/(2*(1+a_th))*(Ugs-Uth)**2 + 0.5*k_clm*(Ugs-Uth)*Uds)

    # print(ID)
    return ID

def calc_ID_gds(Ugs,Uds,Usb):
    """Function to calculate the output characteristics and output conductance
    Input
    ------
    Ugs     : list 
        gate-source voltage.
    Uds     : ndarray 
        drain-source voltage.
    Usb     : float 
        source-substrate voltage.
    Output
    ---------
    ID_out  : ndarray
        Drain current (output characteristics).
    gds     : ndarray
        Output conductance.
    """

    ID_out  = np.zeros((len(Ugs),len(Uds)))
    gds     = np.zeros((len(Ugs),len(Uds)))

    for nUgs in np.arange(np.size(Ugs)):
        for nUds in np.arange(np.size(Uds)):
            ID_out[nUgs,nUds] = calc_ID(Ugs[nUgs],Uds[nUds],Usb)
        gds[nUgs,:] = np.gradient(ID_out[nUgs,:])/np.gradient(Uds)

    return ID_out,gds

def calc_ID_gm(Ugs,Uds,Usb):
    """Function to calculate the transfer characteristics and transconductance
    Input
    ------
    Ugs     : ndarray  
        gate-source voltage.
    Uds     : list
        drain-source voltage.
    Usb     : float 
        source-substrate voltage.
    Output
    ---------
    ID_trans    : ndarray
        Drain current (transfer characteristics).
    gm          : ndarray
        transconductance.
    """

    ID_trans    = np.zeros((len(Uds),len(Ugs)))
    gm          = np.zeros((len(Uds),len(Ugs)))

    for nUds in np.arange(np.size(Uds)):
        for nUgs in np.arange(np.size(Ugs)):
            ID_trans[nUds,nUgs] = calc_ID(Ugs[nUgs],Uds[nUds],Usb)
        gm[nUds,:]  = np.gradient(ID_trans[nUds,:])/np.gradient(Ugs)
    
    return ID_trans,gm

def calc_gmb(Ugs,Uds,Usb):
    """Function to calculate the substrate transconductance
    Input
    ------
    Ugs     : ndarray  
        gate-source voltage.
    Uds     : list
        drain-source voltage.
    Usb     : float 
        source-substrate voltage.
    Output
    ---------
    gmb          : ndarray
        substrate transconductance.
    """
    
    ID_sub    = np.zeros((len(Uds),len(Usb)))
    gmb       = np.zeros((len(Uds),len(Usb)))

    for nUds in np.arange(np.size(Uds)):
        for nUsb in np.arange(np.size(Usb)):
            ID_sub[nUds,nUsb] = calc_ID(Ugs,Uds[nUds],Usb[nUsb])
        gmb[nUds,:]  = np.gradient(ID_sub[nUds,:])/np.gradient(Usb)
    
    return gmb

def update_Uth(*args,**kwargs):
    """Function to update Uth vs Usb plot.
    """
    Usb_op = Usb_slider.value
    Uth = calc_Uth(Uth0,Usb_sweep)
#     print(Uth_line[0][0])
    Uth_line[0][0].set_ydata(Uth)
    
    index = np.argmin(np.abs(Usb_sweep-Usb_op))
    Uth_line[1][0].set_xdata(np.linspace(Usb_op,Usb_op,11))
    Uth_line[1][0].set_ydata(np.linspace(1e-20, Uth[index],11))
    
    ax1_Uth.set_ylim(ymax=np.amax(Uth)*1.25)
    
    fig_Uth.canvas.draw()
    
def update_ID_gds(*args,**kwargs):
    [Ugs_op_min,Ugs_op_max] = Ugs_slider.value
    nOp = (Ugs_op_max-Ugs_op_min)/0.5 + 1
    Ugs_op = np.linspace(Ugs_op_min,Ugs_op_max,4)
    Usb_op = Usb_slider.value
    
    [ID_out,gds]  = calc_ID_gds(Ugs_op,Uds_sweep,Usb_op)
    
    for i in range(len(Ugs_op)):
        ID_out_lines[i][0].set_ydata(ID_out[i]*1e3)
        ID_out_lines[i][0].set_label("{:.2f}".format(round(Ugs_op[i], 2)))
        
        gds_lines[i][0].set_ydata(gds[i]*1e3)
        gds_lines[i][0].set_label("{:.2f}".format(round(Ugs_op[i], 2)))
        
    ax1_ID_gds.legend(loc='upper left',title='$U_{\mathrm{GS}}\mathrm{/V}$')
    ax2_ID_gds.legend(loc='upper right',title='$U_{\mathrm{GS}}\mathrm{/V}$')
    
    ax1_ID_gds.set_ylim(ymax=np.max(ID_out*1.1e3))
    ax2_ID_gds.set_ylim(ymax=np.max(gds*1.1e3))

    fig_ID_gds.canvas.draw()
    
def update_ID_gm_gmb(*args,**kwargs):
    """Function to update transfer characteristics and transconductance plots.
    """
    # get new op points from slider
    [Uds_op_min,Uds_op_max] = Uds_slider.value
    Uds_op = [Uds_op_min,Uds_op_max]
    Usb_op = Usb_slider.value
    
    [ID_trans,gm]  = calc_ID_gm(Ugs_sweep,Uds_op,Usb_op)
    gmb  = calc_gmb(1,Uds_op,Usb_sweep)
    
    for i in range(len(Uds_op)):
        ID_trans_lines[i][0].set_ydata(ID_trans[i]*1e3)
        ID_trans_lines[i][0].set_label("{:.1f}".format(Uds_op[i]))
        
        gm_lines[i][0].set_ydata(gm[i]*1e3)
        gm_lines[i][0].set_label("{:.1f}".format(Uds_op[i]))
        
        gmb_lines[i][0].set_ydata(gmb[i]*1e3)
        gmb_lines[i][0].set_label("{:.1f}".format(Uds_op[i]))
        
    ax1_ID_trans.legend(loc='upper left',title='$U_{\mathrm{DS}}\mathrm{/V}$')
    ax2_gm.legend(loc='upper left',title='$U_{\mathrm{DS}}\mathrm{/V}$')
    ax3_gmb.legend(loc='lower right',title='$U_{\mathrm{DS}}\mathrm{/V}$')
    
    ax1_ID_trans.set_ylim(ymax=np.max(ID_trans*1.1e3))
    ax2_gm.set_ylim(ymax=np.max(gm*1.1e3))
    ax3_gmb.set_ylim(ymin=np.min(gmb*1.1e3))

    fig_ID_gm_gmb.canvas.draw()
    
def update_outTransPlot(*args,**kwargs):
    """Function to update transfer characteristics and transconductance plots.
    """
    update_ID_gds()
    update_ID_gm_gmb()
    
def update_all(*args,**kwargs):
    """Function to update all the plots.
    """
    update_Uth()
    update_outTransPlot()
    
plt.ioff()
plt.grid(True)
    
custom_cycler = (cycler(color=list('rgb')) *
       cycler(linestyle=['-', '--', '-.']))

plt.rc('lines', linewidth=1)
plt.rc('axes', prop_cycle=custom_cycler)
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 1.5
plt.rcParams['grid.color'] = "#cccccc"

figW = 6
figH = 2.25*1.25

### Begin Uth vs Usb and gmb vs Usb fig
# -------------------------------------
fig_Uth = plt.figure(figsize=(figW/2, figH),constrained_layout=True) # 
# gs = fig_Uth_gmb.add_gridspec(1, 1, hspace=0.25, height_ratios=[1], bottom=0.2)
ax1_Uth = fig_Uth.add_subplot(111)
# fig.suptitle('Horizontally stacked subplots')
fig_Uth.canvas.header_visible = False
# fig_Uth.canvas.layout.min_width = '400px'
fig_Uth.canvas.toolbar_visible = True
fig_Uth.canvas.capture_scroll = True

Uth = calc_Uth(Uth0,Usb_sweep)
Usb_op  = 0

# plot Uth and gmb
Uth_line=[]
Uth_line.append(ax1_Uth.plot(Usb_sweep, Uth))
index = np.argmin(np.abs(Usb_sweep-Usb_slider.value))
Uth_line.append(ax1_Uth.plot(np.linspace(Usb_slider.value,Usb_slider.value,11),np.linspace(1e-20, Uth[index],11),color='b',lw=2))
                             
# set legend and label
ax1_Uth.set_xlabel('$U_{\mathrm{SB}}\mathrm{/V}\;→$')
ax1_Uth.set_ylabel('$U_{\mathrm{th}}\mathrm{/V}\;→$')

# set title
ax1_Uth.set_title('$U_{\mathrm{th}}(U_{\mathrm{DS}})$')

# set limits
ax1_Uth.set_xlim(0, np.max(Usb_sweep))
ax1_Uth.set_ylim(ymin=0) #np.amin(Uth)*0.5)

fig_Uth.canvas.draw()
# end Uth vs Usb fig

### Begin output characterisitics and output conductance fig
# ----------------------------------------------------------
fig_ID_gds = plt.figure(figsize=(figW*0.95, figH),constrained_layout=True)
gs = fig_ID_gds.add_gridspec(1, 2, hspace=0.25, wspace=0.25, height_ratios=[1],bottom=0.2)
(ax1_ID_gds, ax2_ID_gds) = gs.subplots(sharex=False, sharey=False)
# fig_ID_gds.suptitle('$I_{\mathrm{D}}(U_{\mathrm{DS}})$'+' und '+'$g_{\mathrm{ds}}(U_{\mathrm{GS}})$')
fig_ID_gds.canvas.header_visible = False
fig_ID_gds.canvas.layout.min_width = '400px'
fig_ID_gds.canvas.toolbar_visible = True
fig_ID_gds.canvas.capture_scroll = True

Ugs_op  = [1,1.5,2,2.5]
Usb_op  = 0

# calculate and plot ID out and gds
[ID_out,gds]  = calc_ID_gds(Ugs_op,Uds_sweep,Usb_op)
ID_out_lines=[]
gds_lines=[]
for i in range(len(Ugs_op)):
    ID_out_lines.append(ax1_ID_gds.plot(Uds_sweep, ID_out[i]*1e3, label=str(Ugs_op[i])))
    gds_lines.append(ax2_ID_gds.plot(Uds_sweep, gds[i]*1e3, label=str(Ugs_op[i])))
    
# set legend and label
ax1_ID_gds.legend(loc='upper left',title='$U_{\mathrm{GS}}\mathrm{/V}$')
ax1_ID_gds.set_xlabel('$U_{\mathrm{DS}}\mathrm{/V}\;→$')
ax1_ID_gds.set_ylabel('$I_{\mathrm{D}}\mathrm{/mA}\;→$')

ax2_ID_gds.legend(loc='upper right',title='$U_{\mathrm{GS}}\mathrm{/V}$')
ax2_ID_gds.set_xlabel('$U_{\mathrm{DS}}\mathrm{/V}\;→$')
ax2_ID_gds.set_ylabel('$g_{\mathrm{m}}\mathrm{/mS}\;→$')


# set title
ax1_ID_gds.set_title('$I_{\mathrm{D}}(U_{\mathrm{DS}})$')
ax2_ID_gds.set_title('$g_{\mathrm{ds}}(U_{\mathrm{DS}})$')

ax1_ID_gds.set_xlim(0, np.max(Uds_sweep))
ax2_ID_gds.set_xlim(0, np.max(Uds_sweep))
ax1_ID_gds.set_ylim(ymin=0)
ax2_ID_gds.set_ylim(ymin=0)
                             
fig_ID_gds.canvas.draw()
# end output characterisitics and output conductance fig

### Begin transfer characterisitics and transconductance fig
# ----------------------------------------------------------
fig_ID_gm_gmb = plt.figure(figsize=(figW*1.5, figH),constrained_layout=True)
gs = fig_ID_gm_gmb.add_gridspec(1, 3, hspace=0.25, wspace=0.25, height_ratios=[1],bottom=0.2)
(ax1_ID_trans, ax2_gm, ax3_gmb) = gs.subplots(sharex=False, sharey=False)
# fig_ID_gm.suptitle('$I_{\mathrm{D}}(U_{\mathrm{GS}})$'+' und '+'$g_{\mathrm{m}}(U_{\mathrm{GS}})$')
fig_ID_gm_gmb.canvas.header_visible = False
fig_ID_gm_gmb.canvas.layout.min_width = '400px'
fig_ID_gm_gmb.canvas.toolbar_visible = True
fig_ID_gm_gmb.canvas.capture_scroll = True

Uds_op = [0.1,3]
Usb_op = 0

# calculate and plot ID and gm
[ID_trans,gm]  = calc_ID_gm(Ugs_sweep,Uds_op,Usb_op)
gmb  = calc_gmb(1,Uds_op,Usb_sweep)
ID_trans_lines=[]
gm_lines=[]
gmb_lines=[]
for i in range(len(Uds_op)):
    ID_trans_lines.append(ax1_ID_trans.plot(Ugs_sweep, ID_trans[i]*1e3, label=str(Uds_op[i])))
    gm_lines.append(ax2_gm.plot(Ugs_sweep, gm[i]*1e3, label=str(Uds_op[i])))
    gmb_lines.append(ax3_gmb.plot(Usb_sweep, gmb[i]*1e3, label=str(Uds_op[i])))

# set legend and label
ax1_ID_trans.legend(loc='upper left',title='$U_{\mathrm{DS}}\mathrm{/V}$')
ax1_ID_trans.set_xlabel('$U_{\mathrm{GS}}\mathrm{/V}\;→$')
ax1_ID_trans.set_ylabel('$I_{\mathrm{D}}\mathrm{/mA}\;→$')

ax2_gm.legend(loc='upper left',title='$U_{\mathrm{DS}}\mathrm{/V}$')
ax2_gm.set_xlabel('$U_{\mathrm{GS}}\mathrm{/V}\;→$')
ax2_gm.set_ylabel('$g_{\mathrm{m}}\mathrm{/mS}\;→$')

ax3_gmb.legend(loc='lower right',title='$U_{\mathrm{DS}}\mathrm{/V}$')
ax3_gmb.set_xlabel('$U_{\mathrm{SB}}\mathrm{/V}\;→$')
ax3_gmb.set_ylabel('$g_{\mathrm{mb}}\mathrm{/mS}\;→$')

# set title
ax1_ID_trans.set_title('$I_{\mathrm{D}}(U_{\mathrm{GS}})$')
ax2_gm.set_title('$g_{\mathrm{m}}(U_{\mathrm{GS}})$')
ax3_gmb.set_title('$g_{\mathrm{mb}}(U_{\mathrm{SB}})$')

# set limits
ax1_ID_trans.set_xlim(0, np.max(Ugs_sweep))
ax1_ID_trans.set_ylim(ymin=0)

ax2_gm.set_xlim(0, np.max(Ugs_sweep))
ax2_gm.set_ylim(ymin=0)

ax3_gmb.set_xlim(0, np.max(Usb_sweep))
ax3_gmb.set_ylim(ymax=0.01)

# draw figure
fig_ID_gm_gmb.canvas.draw()
# end transfer characterisitics and transconductance fig


# observe sliders
# ---------------
dox_slider.observe(update_all)
NA_slider.observe(update_all)
Usb_slider.observe(update_all)

W_slider.observe(update_outTransPlot)

Ugs_slider.observe(update_ID_gds)
Uds_slider.observe(update_ID_gm_gmb)


# Layout definition
fixedPara_layout = Layout(
    flex_flow       = 'column',
    align_items   = 'stretch',
    width         = '250px'
)
parameter_layout = Layout(
    align_items   = 'baseline',
    width         = '100%',
    flex_grow       = 1
)

largeVBox_layout = Layout(
    align_items     = 'center',
    width         = '75%',
)

smallVBox_layout = Layout(
    align_content     = 'flex-end',
    width         = '27%',
)

center_layout = Layout(
    display         = 'flex',
    flex_flow       = 'column',
    align_items     = 'center',
    width           = '100%',
    flex_grow       = 1,
    justify_content = 'space-around'
)

final_layout = VBox([
    HBox([
        VBox([
                HBox([Label('$Gleichungen$')]),
                HBox([IDeqn_title]),
                HBox([IDeqn_lin]),
                HBox([IDeqn_sat]),
                HBox([kn_ath0_gamma_Cox]),
                HBox([Uth_kclm_Udssat]),
            ],layout=Layout(width='60%',align_items   = 'stretch',flex_grow       = 1,justify_content = 'space-around')),
        VBox([
                HBox([fig_Uth.canvas])
            ],layout=Layout(width='40%')),
        ]), # , layout=fixedPara_layout
    HBox([
        VBox([
                VBox([
                    HBox([Label('$Veränderbare\;parameters$')]),
                    HBox([W_slider]), 
                    HBox([dox_slider]),
                    HBox([NA_slider]),
                    HBox([Usb_slider]),
                ]), # , layout=fixedPara_layout
                VBox([
                    HBox([Label('$Feste\;parameters$')]),
                    HBox([Uth0_wid]),
                    HBox([Lch_wid]),
                    HBox([mu_n_wid]),
                    HBox([ni_wid]),
                    HBox([UA_wid]),
                    HBox([temp_wid]),
                ]), # , layout=fixedPara_layout

            ],layout=Layout(width='32%')), # , layout=smallVBox_layout
        VBox([
                HBox([fig_ID_gds.canvas]),
                HBox([Ugs_slider], layout=center_layout)
            ],layout=Layout(width='68%')),
        ]),
    HBox([
        VBox([
            HBox([fig_ID_gm_gmb.canvas]),
            HBox([Uds_slider], layout=center_layout)
        ])
    ])
    
])

final_layout

# IDeqn_sat.style.keys